# Building your own algorithm container 

## [(원본)](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb)

Amazon SageMaker을 사용하면 SageMaker환경에서 훈련하고 배포할 수 있도록 자신의 알고리즘을 패키징할 수 있습니다. 
이 노트북은 SageMaker에서 Docker 컨테이너를 빌드하고 훈련 및 추론을 사용하는 방법에 대한 예제를 제공할 것입니다. 

컨테이너에 알고리즘을 패키징하면 프로그램 언어, 환경, 프레임워크 혹은 의존성과는 상관없이, 거의 모든 코드를 Amazon SageMaker환경으로 가져올 수 있습니다. 


_**Note:**_ SageMaker는 현재 [pre-built scikit container](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/scikit_learn_iris/Scikit-learn%20Estimator%20Example%20With%20Batch%20Transform.ipynb)를 포함하고 있습니다.  우리는 scikit 알고리즘이 필요한 대부분의 모든 경우에 pre-built container를 사용하기를 권장합니다. 그러나 이 예제는 자신만의 컨테이너를 통해 다른 라이브러리들을 SageMaker로 가져오기 위한 아웃라인으로서 제공합니다. 



1. [Building your own algorithm container](#Building-your-own-algorithm-container)
  1. [언제 자신만의 알고리즘 컨테이너를 만들어야만 할까요?](#언제-자신만의-알고리즘-컨테이너를-만들어야만-할까요%3F)
  1. [권한](#권한)
  1. [예제](#예제)
  1. [프리젠테이션](#프리젠테이션)
1. [파트 1: Amazon SageMaker와 함께 사용할 알고리즘 패키징과 업로드](#파트-1%3A-Amazon-SageMaker와-함께-사용할-알고리즘-패키징과-업로드)
    1. [Docker 개요](#Docker-개요)
    1. [Amazon SageMaker가 Docker container를 실행하는 방법](#Amazon-SageMaker가-Docker-container를-실행하는-방법)
      1. [Running your container during training](#Running-your-container-during-training)
        1. [The input](#The-input)
        1. [The output](#The-output)
      1. [Running your container during hosting](#Running-your-container-during-hosting)
    1. [샘플 컨테이너 파트](#샘플-컨테이너-파트)
    1. [Dockerfile](#Dockerfile)
    1. [컨테이너 빌드 및 등록](#컨테이너-빌드-및-등록)
  1. [로컬 머신이나 Amazon SageMaker 노트북 인스턴스에서 알고리즘 테스트하기](#로컬-머신이나-Amazon-SageMaker-노트북-인스턴스에서-알고리즘-테스트하기)
1. [파트 2: Amazon SageMaker에서 자신의 알고리즘 사용하기](#파트-2%3A-Amazon-SageMaker에서-자신의-알고리즘-사용하기)
  1. [환경 설정](#환경-설정)
  1. [세션 생성](#세션-생성)
  1. [훈련을 위한 데이터 업로드](#훈련을-위한-데이터-업로드)
  1. [Estimator 생성 및 모델 fit 하기](#Estimator-생성-및-모델-fit-하기)
  1. [모델 호스팅하기](#모델-호스팅하기)
    1. [모델 배포하기](#모델-배포하기)
    2. [일부 데이터를 선택하고 예측에 사용하기](#일부-데이터를-선택하고-예측에-사용하기)
    3. [선택적 정리](#선택적-정리)
  1. [배치 변환 Job 실행](#배치-변환-Job-실행)
    1. [변환-Job-생성하기](#Create-a-Transform-Job)
    2. [출력-보기](#View-Output)

_or_ I'm impatient, just [let me see the code](#The-Dockerfile)!

## 언제 자신만의 알고리즘 컨테이너를 만들어야만 할까요?

Amazon SageMaker에 자신의 코드를 가져와서 컨테이너를 생성할 필요는 없을수도 있습니다. SageMaker에세 제공하는 Apache MXNet이나 TensorFlow와 같은 프레임워크를 사용할때, 프레임워크에서 제공하는 SDK entry points를 사용하여 알고리즘을 구현하는 Python 코드를 간단히 사용할 수 있습니다. 
이 프레임워크들의 세트들은 지속적으로 확장하고 있기 때문에, 자신의 알고리즘이 일반적인 머신러닝환경에서 작성된 경우 최근의 지원 리스트를 확인하는 것을 권장합니다. 

사용자 환경이나 프레임워크를 위한 SDK의 직접적인 지원이 있더라고 자신만의 컨테이너를 만드는 것이 더 효과적일 수도 있습니다. 
자신의 알고리즘의 구현하는 코드가 자체적으로 매우 복잡하거나 프레임워크에 특별한 추가가 필요할 경우에는 자신만의 컨테이너를 만드는 것이 더 좋을 선택일 수 있습니다. 

사용자 환경을 직접적으로 지원하는 SDK가 없더라도 걱정할 필요가 없습니다. 이 과정을 통해서 자신만의 컨테이너를 만드는 것이 매우 간단하는 것을 알 수 있을 것입니다. 

## 권한

이 노트북을 실행하기 위해서는 일반적인 "SageMakerFullAccess"권한 외에도 다른 권한이 필요합니다. 이것은 Amazon ECR에 신규 레파지토리를 생성해야하기 때문입니다. 이 권한을 추가하는 가장 쉬운 방법은 노트북 인스턴스를 시작할 때 사용했던 Role에 Managed Policy인`AmazonEC2ContainerRegistryFullAccess`를 추가하는 것입니다. 이 작업을 수행할 때 노트북 인스턴스를 재시작할 필요는 없으며 새로운 권한은 즉시 반영이 됩니다. 

## 예제

여기서는 널리 사용되는 [scikit-learn][] 머신러닝 패키지에서 [decision tree][] 알고리즘을 보여주는 간단한 Python에제를 패키징하는 방법을 보여줍니다.
이 예제는 Amazon SageMaker에서 자신의 코드를 훈련하고 호스팅할 수 있게 하는 구조를 보여주기 위한 것으로서, 매우 심플합니다. 

여기서 보여지는 아이디어들은 어떠한 언어나 환경에서도 작동합니다. 사용자는 추론을 위한 HTTP 요청들을 처리하는 환경을 위해 적합한 툴을 선택할 필요가 있습니다. 그러나 요즘에는 모든 언어에서 좋은 HTTP 환경을 제공하고 있습니다. 

이 예제에서 훈련과 호스팅을 지원하기 위해서 단일 이미지를 사용합니다. 우리는 오직 하나의 이미지만 관리하고 이것으로 모든것을 할 수 있도록 설정할 수 있기 때문에 매우 간단합니다. 때로는 각각 다른 요구사항으로 인해 훈련과 호스팅을 위해 이미지를 분리하기를 원할 수도 있습니다. 아래에서 설명한 부분들을 별도의Dockerfile로 나누고 두개의 이미지를 만드시기 바랍니다. 개발과 관리를 좀 더 편리하게 하기 위해서는 한 개 혹은 두 개의 이미지를 선택하는 것은 매우 중요합니다. 


훈련이나 호스팅을 위해서 Amazon SageMaker만을 사용하고 있는 경우, 자신의 컨테이너에 사용하지 않는 기능을 만들 필요는 없습니다. 


[scikit-learn]: http://scikit-learn.org/stable/
[decision tree]: http://scikit-learn.org/stable/modules/tree.html

## 프리젠테이션


이 프리젠테이션은 _building_ 컨테이너와 _using_ the container 의 두 파트로 나뉩니다. 

# 파트 1: Amazon SageMaker와 함께 사용할 알고리즘 패키징과 업로드


### Docker 개요

Docker에 익숙하다면 다음 섹션을 건너띄어도 됩니다. 


많은 데이터 과학자들에게는 Docker 컨테이너가 새로운 개념이지만, 여기에서 볼 수 있듯이 어렵지 않습니다. 

Docker는 임의의 코드를 완전히 독립적인 _이미지_로 패키지하는 간단한 방법을 제공합니다. 이미지가 있으면 Docker를 사용하여 해당 이미지를 기반으로 _컨테이너_를 실행할 수 있습니다. 컨테이너를 실행하는 것은 컨테이너가 프로그램을 실행하기위한 완전히 독립된 환경을 생성한다는 점을 제외하고 머신에서 프로그램을 실행하는 것과 같습니다. 컨테이너는 서로 호스트 환경과 분리되어 있으므로 프로그램을 설정하는 방법은 실행 위치에 관계없이 프로그램이 실행되는 방식입니다.

Docker는 (a)언어에 독립적이며 (b)시작 명령, 환경 변수 등 전체 운영 환경을 포함하므로 conda 또는 virtualenv와 같은 환경 관리자보다 강력합니다.

어떤 면에서 Docker 컨테이너는 가상 머신과 비슷하지만 훨씬 가볍습니다. 예를 들어, 컨테이너에서 실행되는 프로그램은 1초 이내에 시작할 수 있으며 많은 컨테이너가 동일한 실제 머신 또는 가상 머신 인스턴스에서 실행될 수 있습니다.

Docker는 `Dockerfile`이라는 간단한 파일을 사용하여 이미지가 어셈블되는 방식을 지정합니다. 아래에서 그 예를 볼 수 있습니다. 자신이나 다른 사람이 만든 Docker 이미지를 기반으로 Docker 이미지를 만들 수 있으므로 작업이 약간 단순화됩니다.

Docker는 프로그래밍 및 실행 영역에서 유연성과 잘 정의 된 코드 사양으로 인해 프로그래밍 및 개발자 커뮤니티에서 매우 인기가 있습니다. [Amazon ECS]와 같이 지난 몇 년간 구축된 많은 서비스의 토대가 되고 있습니다.


Amazon SageMaker는 Docker를 사용하여 사용자가 임의의 알고리즘을 훈련하고 배포할 수 있도록 합니다. 

Amazon SageMaker에서는 Docker 컨테이너가 훈련을 위해 수행하는 특정한 방법이 있고 호스팅에서도 다른 방법을 사용합니다. 다음 섹션에서는 SageMaker 환경을 위해 컨테이너를 빌드하는 방법에 대해 간략하게 설명합니다.

유용한 링크:

* [Docker home page](http://www.docker.com)
* [Getting started with Docker](https://docs.docker.com/get-started/)
* [Dockerfile reference](https://docs.docker.com/engine/reference/builder/)
* [`docker run` reference](https://docs.docker.com/engine/reference/run/)

[Amazon ECS]: https://aws.amazon.com/ecs/

### Amazon SageMaker가 Docker container를 실행하는 방법

훈련 또는 호스팅에서 동일한 이미지를 실행할 수 있기 때문에, Amazon SageMaker는 `train` 이나 `serve` 인수와 함께 컨테이너를 실행합니다. 
컨테이너에서 이 인수를 처리하는 방법은 컨테이너에 따라 다릅니다:

* 이 예제에서 Dockerfile안에 `ENTRYPOINT`를 정의하지 않습니다. 따라가 Docker는 훈련 시간에는 `train`명령을, 서비스 시간에는 `serve`명령을 실행합니다. 이 예제에서 우리는 실행가능한 Python script들을 정의하지만, 이것들은 우리가 해당 환경에서 시작할 수 있는 모든 프로그램이 될 수 있습니다.

* Dockerfile의 `ENTRYPOINT` 에 프로그램을 지정한다면, 그 프로그램은 시작시점에 실행되고 그것의 첫번째 인자는 `train`이나 `serve`가 될것입니다. 프로그램은 인자를 보고 무엇을 할 지 결정할 수 있습니다. 

* 훈련과 호스팅을 위해 별도의 컨테이너를 생성한다면 (혹은 하나만 생성한다면), DockerFile의 `ENTRYPOINT`에 프로그램을 정의하고, 첫번째인자를 무시 (혹은 검증)하게 할 수도 있습니다.  

#### Running your container during training

Amazon SageMaker가 훈련을 실행할 때, `train` 스크립트는 일반적인 Python 프로그램과 같이 실행됩니다. 사용을 위해서는 `/opt/ml` 디렉토리 아래에 많은 파일들이 배치되어야 합니다. 

    /opt/ml
    ├── input
    │   ├── config
    │   │   ├── hyperparameters.json
    │   │   └── resourceConfig.json
    │   └── data
    │       └── <channel_name>
    │           └── <input data>
    ├── model
    │   └── <model files>
    └── output
        └── failure

##### The input

* `/opt/ml/input/config`는 프로그램을 실행 방법을 제어하기 위한 정보를 포함하고 있습니다. `hyperparameters.json`는 하이퍼파라미터의 이름과 값이 저장하는 JSON 형식의 Dictionary입니다. 이 값들은 모두 문자열이어야 하므로, 값들을 변환해야 할 수도 있습니다. 
`resourceConfig.json`은 분산 훈련에서 사용하는 네트워크 레이아웃을 설명하기 위한  JSON 형식의 파일입니다. scikit-learn은 분산 훈련을 지원하지 않으므로, 여기에서는 이것을 무시합니다. 

* `/opt/ml/input/data/<channel_name>/` (for File mode)는 해당 채널의 입력 데이터를 포함합니다. 채널은 CreateTrainingJob를 호출할 때 생성이 되지만, 일반적으로 채널이 알고리즘이 예상하는 것과 일치하는 것이 중요합니다. 각 채널의 파일들은 S3로부터 이 디렉토리로 복사되고 S3 Key구조로 표시된 트리 구조를 유지합니다. 

* `/opt/ml/input/data/<channel_name>_<epoch_number>` (for Pipe mode)는 주어진 epoch을 위한 pipe 입니다. Epoch은 0에서 시작하여 읽을 때마다 하나씩 올라갑니다. 실행할 수 있는 epoch의 수는 제한이 없지만, 다음 epoch을 읽기 전에는 각 pipe를 닫아야 합니다. 



##### The output

* `/opt/ml/model/`는 알고리즘이 생성한 모델을 쓰는 디렉토리입니다. 모델은 당신이 원하는 어떤 형식이든 될 수 있습니다. 그것은 단일 파일 혹은 전체 디렉토리 트리일 수도 있니다. SageMaker는 이 디렉토리안의 어떤 파일이든 tar로 압축 파일을 만들어 패키징합니다. 이 파일은 `DescribeTrainingJob` 결과에서 리턴한 S3위치에서 사용할 수 있습니다. 

* `/opt/ml/output`는 알고리즘이 Job 실패이유를 설명하는 `failure` 파일을 작성하기 위한 디렉토리입니다. 이 파일의 내용은 `DescribeTrainingJob`의 `FailureReason` 필드 리턴됩니다. 성공한 Job은 이 파일을 쓸 이유가 없으므로 무시됩니다. 


#### Running your container during hosting

호스팅은 HTTP를 통해 들어오는 추론 요청을 응답하기 때문에 훈련과는 매우 다른 모델입니다. 이 예제에서, 우리는 Python serving 스택을 사용하여 강력하고 확장 가능한 추론 요청 서비스를 제공합니다: 

![Request serving stack](stack.png)


이 스택은 샘플 코드에서 구현되었고 대부분 그냥 둡니다.

Amazon SageMaker는 컨테이너에서 두개의 URL을 사용합니다: 

* `/ping` 는 인프라로부터  `GET` 요청을 받습니다. 컨테이너가 가동되고 요청을 받아들이면  프로그램은 200을 리턴합니다. 

* `/invocations`는 클라이언트의 추론 `POST` 요청을 받는 엔드포인트입니다. 요청과 응답의 형식은 알고리즘에 따라 다릅니다. 클라이언트는  `ContentType`와 `Accept` 헤더를 제공한 경우, 이것들도 역시 전달이 됩니다.

컨테이터는 훈련하는 동안 작성된 것과 같은 장소에 모델 파일이 저장됩니다:

    /opt/ml
    └── model
        └── <model files>



### 샘플 컨테이너 파트

`container` 디렉토리에는 Amazon SageMaker에서 샘플 알고리즘을 패키지할 때 필요한 모든 구성요소가 있습니다:

    .
    ├── Dockerfile
    ├── build_and_push.sh
    └── decision_trees
        ├── nginx.conf
        ├── predictor.py
        ├── serve
        ├── train
        └── wsgi.py

각 항목에 대해 차례로 얘기해 보도록 하겠습니다:

* __`Dockerfile`__ 는 Docker 컨테이너 이미지를 생성하는 방법에 대해 기술합니다. 자세한 내용을 아래를 참조하세요.
* __`build_and_push.sh`__  는 Dockerfile을 사용하여 컨테이너 이미지를 생성하고 ECR로 이것을 푸시하는 스크립트입니다. 이 노트북의 뒷부분에서 명령을 직접 호출하지만, 자신의 알고리즘에 맞게 복사하고 실행할 수 있습니다. 
* __`decision_trees`__ 는 컨테이너에 설치될 파일들을 포함하는 디렉토리입니다. 
* __`local_test`__ 는 Amazon SageMaker 노트북 인스턴스를 포함한 Docker를 실행할 수 있는 어떠한 컴퓨터에서라도 새로운 컨테이너를 테스트할 수 있는 방법을 보여주는 디렉토리입니다. 이 방법을 통해 Amazon SageMaker와 함께 컨테이너를 사용하기 전에, 작은 데이터셋을 신속하게 반복적으로 사용하여, 구조적인 버그를 제거할 수 있습니다. 이 노트북의 뒷 부분에서 로컬 테스트를 진행합니다. 

이 간단한 어플리케이션은 컨테이너에 5개의 파일만 설치합니다. 그 정도만 필요할 수도 있고 또는 많은 루틴이 있는 경우라면, 더 많이 설치할 수도 있습니다. 이 5 개는 Python 컨테이너의 표준 구조를 보여주지만, 다른 툴셋을 자유롭게 선택할 수 있으므로 다른 구조를 가질 수 있습니다. 다른 프로그래밍 언어로 작성했다면, 선택한 프레임 워크 및 도구에 따라 구조가 달라집니다.


컨테이너 안에 넣어야 할 파일들은 다음과 같습니다: 

* __`nginx.conf`__ 는 nginx 프론트엔드의 Configuration 파일입니다. 일반적으로이 파일을 있는 그대로 사용할 수 있습니다.
* __`predictor.py`__ 는 실제로 Flask 웹 서버와  앱의 decision tree 예측을 실제로 구현하는 프로그램입니다. 실제 앱의 예측 부분을 수정하길 원할것입니다. 이 알고리즘은 단순하기 때문에, 우리는 이 파일에서 모든 처리를 수행하지만, 사용자 정의 로직 구현을 위해 파일을 별도로 분리할 수도 있습니다. 
* __`serve`__ 는 컨테이너가 호스팅을 시작할 때 시작하는 프로그램입니다. `predictor.py`에서 정의된 Flask 앱의 여러 인스턴스를 실행하는 gunicorn 서버를 시작합니다. 이 파일은 있는 그대로 가져갈 수 있을 것입니다. 
* __`train`__ 는 훈련을 위해 컨테이너가 실행될 때 호출되는 프로그램입니다. 훈련 알고리즘을 구현하기 위해 이 프로그램을 수정합니다. 
* __`wsgi.py`__ 는 Flask app을 호출하기 위한 작은 래퍼입니다. 이 파일은 있는 그대로 사용할 수 있습니다. 
 

요약하면, 어플리케이션에서 변경하려는 두 파일은  `train`와 `predictor.py` 입니다. 

### Dockerfile

Dockerfile은 빌드하려는 이미지를 설명합니다. 실행하려는 시스템의 전체 운영 체제 설치를 설명하는 것과 같은 것으로 생각할 수 있습니다. Docker 컨테이너는 기본 운영을 위해 호스트 시스템의 Linux를 사용하기 때문에 전체 운영 체제보다 조금 가볍습니다.

파이썬 과학 스택의 경우 표준 우분투 설치에서 시작하여 일반 도구를 실행하여 scikit-learn에 필요한 것을 설치합니다. 마지막으로 특정 알고리즘을 구현하는 코드를 컨테이너에 추가하고 적절한 환경을 설정합니다.

그 과정에서 추가 공간을 정리합니다. 이렇게하면 컨테이너가 더 작고 빠르게 시작됩니다.

예를 들어 Dockerfile을 살펴 보도록 하겠습니다:

In [5]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:16.04

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.
RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py && \
    pip install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gevent gunicorn && \
        (cd /usr/local/lib/python2.7/dist-packages/scipy/.libs; rm *; ln ../..

### 컨테이너 빌드 및 등록

다음 쉘 코드는`docker build`를 사용하여 컨테이너 이미지를 작성하고`docker push`를 사용하여 컨테이너 이미지를 ECR에 푸시하는 방법을 보여줍니다. 이 코드는 쉘 스크립트`container/build-and-push.sh`로도 사용 가능하며`build-and-push.sh decision_trees_sample`으로 실행하여 이미지`decision_trees_sample`을 빌드 할 수 있습니다.

이 코드는 사용중인 계정과 현재 기본 리전 (SageMaker 노트북 인스턴스를 사용하는 경우 노트북 인스턴스가 생성 된 리전)에서 ECR Repository를 찾습니다. Repository를가 존재하지 않으면 스크립트가 이를 생성합니다.



In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-decision-trees

cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

## 로컬 머신이나 Amazon SageMaker 노트북 인스턴스에서 알고리즘 테스트하기

Amazon SageMaker로 알고리즘을 처음 패키징하는 동안, 알고리즘이 올바르게 작동하는지 직접 테스트하고 싶을 것입니다. `container/local_test` 디렉토리에는 이를 위한 프레임 워크가 있습니다. 컨테이너를 실행하고 사용하기 위한 3 개의 쉘 스크립트와 위에서 설명한 것과 유사한 디렉토리 구조가 포함되어 있습니다

스크립트는 다음과 같습니다:

* `train_local.sh`: 이미지 이름과 이것을 함께 실행하면 로컬 트리에 대한 훈련이 실행됩니다. 예를 들어`$./train_local.sh sagemaker-decision-trees`를 실행할 수 있습니다. 그것은 `/test_dir/model` 디렉토리에 모델을 생성합니다. 알고리즘에 대한 올바른 채널 및 데이터로 설정되도록 `test_dir/ input/data/...` 디렉토리를 수정해야 합니다. 또한 테스트하려는 하이퍼파라미터 설정(문자열)을 위해 `input/config/hyperparameters.json` 파일을 수정해야 합니다. 

* `serve_local.sh`: 모델을 훈련한 후 이미지 이름과 함께 실행하면 모델을 서빙할 것입니다. 예를 들어`$./serve_local.sh sagemaker-decision-trees`를 실행할 수 있습니다. 이것은 실행되고 요청을 기다립니다. 중단을 위해 키보드 인터럽트를 사용할 수 있습니다.


* `predict.sh`: 페이로드 파일의 이름과 원하는 HTTP Content Type(옵션)으로 이를 실행하시기 바랍니다. Content Type은 기본적으로`text/csv`입니다. 예를 들면 `$./predict.sh payload.csv text/csv`를 실행할 수 있습니다


이 디렉토리는 여기에 제시된 의사결정트리 샘플 알고리즘을 테스트하도록 설정되었습니다.

# 파트 2: Amazon SageMaker에서 자신의 알고리즘 사용하기

패키징된 컨테이너를 가지게 되었으면, 이 컨테이너를 사용하여 모델을 훈련하고 모델을 호스팅 또는 배치변환을 위해 사용할 수 있습니다. 위에서 만든 알고리즘으로 그렇게 진행해 보겠습니다. 

## 환경 설정

여기서는 사용할 Bucket과 SageMaker 작업에 사용될 Role을 지정합니다.

In [21]:
# S3 prefix
bucket = '<your_S3_bucket_name_here>'
prefix = 'DEMO-scikit-byo-iris'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## 세션 생성

세션은 SageMaker에 대한 연결 파라미터들을 기억합니다. 이를 사용하여 모든 SageMaker 작업을 수행합니다.

In [8]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## 훈련을 위한 데이터 업로드

방대한 양의 데이터로 대규모 모델을 훈련할 때는 일반적으로 Amazon Athena, AWS Glue 또는 Amazon EMR과 같은 빅 데이터 도구를 사용하여 S3에서 데이터를 생성합니다. 이 예제의 목적을 위해 우리는 고전적인 [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set)을 사용합니다.

SageMaker Python SDK에서 제공하는 도구를 사용하여 데이터를 기본 버킷에 업로드 할 수 있습니다.


In [9]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

## Estimator 생성 및  모델 fit 하기

알고리즘에 맞게 SageMaker를 사용하기 위해, 컨테이너를 사용하여 훈련하는 방법을 정의하는 'Estimator'를 생성합니다. 여기에는 SageMaker 훈련을 호출하는 데 필요한 구성이 포함됩니다: 

* The __container name__. 이것은 위의 쉘 명령에서 생성이 되었습니다.
* The __role__. 위에서 정의한 바와 같습니다.
* The __instance count__ 훈련에 사용할 머신의 수를 지정합니다.
* The __instance type__ 훈련에 사용할 머신의 유형을  지정합니다.
* The __output path__ model artifact가 작성될 위치를 결정합니다. 
* The __session__  위에서 정의한 SageMaker session object 입니다.

다음으로 estimator에서 fit() 사용하여 우리가 위에서 업로드한 데이터를 훈련합니다. 

In [10]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-decision-trees:latest'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

2019-11-21 09:26:24 Starting - Starting the training job...
2019-11-21 09:26:40 Starting - Launching requested ML instances......
2019-11-21 09:27:47 Starting - Preparing the instances for training...
2019-11-21 09:28:25 Downloading - Downloading input data
2019-11-21 09:28:25 Training - Downloading the training image...
2019-11-21 09:28:47 Training - Training image download completed. Training in progress.Starting the training.
Training complete.

2019-11-21 09:29:11 Uploading - Uploading generated training model
2019-11-21 09:29:11 Completed - Training job completed
Training seconds: 52
Billable seconds: 52


## 모델 호스팅하기

훈련된 모델을 사용하여 HTTP 엔드포인트로 실시간 예측을 얻을 수 있습니다. 다음 단계에 따라 프로세스를 진행하십시오.

### 모델 배포하기

SageMaker 호스팅에 모델을 배포하려면 피팅된 모델에 대한 'deploy' 호출만 있으면 됩니다. 이 호출은 인스턴스 수, 인스턴스 유형 및 선택적으로 serializer 및 deserializer 기능을 사용합니다. 이것은 최종 predictor가 엔드포인트에서 생성할 때 사용됩니다.

In [11]:
from sagemaker.predictor import csv_serializer
predictor = tree.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

---------------------------------------------------------------------------------------------------!


### 일부 데이터를 선택하고 예측에 사용하기

몇 가지 예측을 수행하기 위해 훈련에 사용했던 일부 데이터를 추출하고 이에 대한 예측을 수행합니다. 물론 이것은 잘못된 통계 관행이지만 메커니즘이 어떻게 작동하는지 알 수 있는 좋은 방법입니다.


In [12]:
shape=pd.read_csv("data/iris.csv", header=None)
shape.sample(3)

,0,1,2,3,4
149,virginica,5.9,3.0,5.1,1.8
109,virginica,7.2,3.6,6.1,2.5
22,setosa,4.6,3.6,1.0,0.2


In [13]:
# drop the label column in the training set
shape.drop(shape.columns[[0]],axis=1,inplace=True)
shape.sample(3)

,1,2,3,4
31,5.4,3.4,1.5,0.4
62,6.0,2.2,4.0,1.0
14,5.8,4.0,1.2,0.2


In [14]:
import itertools

a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data=shape.iloc[indices[:-1]]


예측은 deploy에서 얻은 predictor와 예측기위한 데이터를 사용하여 예측을 호출하는 것으로 매우 쉽습니다. serializers는 데이터 변환을 담당합니다.

In [15]:
print(predictor.predict(test_data.values).decode('utf-8'))

setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica



### 선택적 정리
엔드포인트가 끝나면, 그것을 정리해야 합니다. 

In [16]:
sess.delete_endpoint(predictor.endpoint)

## 배치 변환 Job 실행

[Amazon SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)를 사용하면 대용량 데이터 세트에 대한 추론을 얻을 수 있습니다. 배치 변환 Job은 input 데이터 S3 위치를 가져와서 지정된 S3 output 폴더에 예측을 출력합니다. 호스팅과 마찬가지로 훈련 데이터에 대한 추론을 추출하여 배치 변환을 테스트할 수 있습니다.

### 변환 Job 생성하기

컨테이너를 사용하여 데이터셋에 대한 추론 결과를 얻기위한 방법을 정의하는 'Transformer'를 생성합니다. 여기에는 SageMaker 배치 변환을 호출하는 데 필요한 구성이 포함됩니다.


* The __instance count__ 추론을 추출하기 위해 사용하는 머신의 수
* The __instance type__ 추론을 추출하기 위해 사용하는 머신의 유형
* The __output path__ 추론 결과가 쓰여질 위치를 결정

In [ ]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = tree.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path,
                               assemble_with='Line',
                               accept='text/csv')

transformer의 tranform()을 사용하여 업로드한 데이터에 대한 추론 결과를 얻습니다. transformer를 호출할 때 이 옵션을 사용할 수 있습니다.


* The __data_location__ 입력 데이터의 위치
* The __content_type__ 컨테이너에 HTTP 요청을 할 때 설정된 Content Type
* The __split_type__ 입력 데이터를 분할하기 위한 구분자
* The __input_filter__ 컨테이너에 HTTP 요청을하기 전에 입력의 첫 번째 열 (ID)이 삭제됨

In [18]:
transformer.transform(data_location, content_type='text/csv', split_type='Line', input_filter='$[1:]')
transformer.wait()

..................Starting the inference server with 4 workers.
[2019-11-21 09:57:45 +0000] [11] [INFO] Starting gunicorn 19.9.0
[2019-11-21 09:57:45 +0000] [11] [INFO] Listening at: unix:/tmp/gunicorn.sock (11)
[2019-11-21 09:57:45 +0000] [11] [INFO] Using worker: gevent
[2019-11-21 09:57:45 +0000] [16] [INFO] Booting worker with pid: 16
[2019-11-21 09:57:45 +0000] [17] [INFO] Booting worker with pid: 17
[2019-11-21 09:57:45 +0000] [18] [INFO] Booting worker with pid: 18
[2019-11-21 09:57:45 +0000] [19] [INFO] Booting worker with pid: 19

169.254.255.130 - - [21/Nov/2019:09:58:23 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [21/Nov/2019:09:58:23 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Invoked with 150 records
169.254.255.130 - - [21/Nov/2019:09:58:23 +0000] "POST /invocations HTTP/1.1" 200 1400 "-" "Go-http-client/1.1"
169.254.255.130 - - [21/Nov/2019:09:58:23 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1

추가적인 설정 옵션 정보는 [CreateTransformJob API](https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateTransformJob.html)을 참고하시기 바랍니다

### 출력 보기 
S3에서 위의 변환Job의 결과를 읽고 출력합니다.

In [20]:
s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/iris.csv.out".format(transform_output_folder), '/tmp/iris.csv.out')
with open('/tmp/iris.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))

Transform results: 
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica
